# Алгоритм Кронекера и численно-аналитический метод

Задача. Дан многочлен $f \in \mathbb{Z}[x]$, трубется выяснить, является ли он простым.
## Алгоритм Кронекера 

In [1]:
import itertools
def factors(n):
    ans=[]
    for (p,m)  in ZZ(abs(n)).factor():
        ans=ans+[p for mm in range(m)]
    return [(-1)^k*prod(a) for a in Combinations(ans) for k in [0,1]]

def ipoly(points,x=x):
    m=1
    f=0
    for (xx,yy) in points:
        f=f+ (yy-f.subs([x==xx]))*m/m.subs([x==xx])
        m=m*(x-xx)
    return f

In [2]:
def Kronecker(f):
    n=f.degree()
    m=floor(n/2)
    L=[]
    ans=[]
    for k in range(m+1):
        a=f(k)
        if a==0:
            ans=[x-k, f.quo_rem(x-k)[0]]
            break
        else:
            L.append(factors(a))
    L=iter.product(*L)
    while ans==[]:
        try:
            l=next(L)
            points=zip(range(m+1),l)
            g=QQ[x](ipoly(points))
            if g.degree()>0:
                [u,r]=f.quo_rem(g)
                if r==0:
                    ans=[g,u]
        except StopIteration as err:
            break
    return ans

1.) Как исправить функцию Kronecker, чтобы она работа при любом выборе имени незиависимой переменной?

In [3]:
var("t")
f=expand(QQ[t](t^3+1))
print(f)
Kronecker(f)

t^3 + 1


AttributeError: 'builtin_function_or_method' object has no attribute 'product'

Указание. Воспользуйтесь двумя методами:

In [4]:
f.parent()

Univariate Polynomial Ring in t over Rational Field

In [5]:
f.variables()

(t,)

In [6]:
def Kronecker(f):
    x=f.variables()[0]
    K=f.parent()
    n=f.degree()
    m=floor(n/2)
    ans=[]
# Цикл 1
    L=[]
    for k in range(m+1):
        a=f(k)
        if a==0:
            ans=[x-k, f.quo_rem(K(x-k))[0]]
            break
        else:
            L.append(factors(a))
    L=itertools.product(*L)
# Цикл 2
    while ans==[]:
        try:
            l=next(L)
            points=zip(range(m+1),l)
            g=K(ipoly(points,x=SR(x)))
            if g.degree()>0:
                [u,r]=f.quo_rem(g)
                if r==0:
                    ans=[g,u]
        except StopIteration as err:
            ans = 'poly is prime'
            break
    return ans

In [8]:
var("t")
f=expand(QQ[t](t^3+1))
print(f)
Kronecker(f)

t^3 + 1


[t + 1, t^2 - t + 1]

In [8]:
var("t")
f=expand(QQ[t](t^3+t+1))
print(f)
Kronecker(f)

t^3 + t + 1
(1, 1)
(1, -1)
(1, 3)
(1, -3)
(-1, 1)
(-1, -1)
(-1, 3)
(-1, -3)


'poly is prime'

2.) Почему используется конструкция ipoly(points,x=SR(x))? Что означает SR?

Мы используем параметр x=SR(x), так как функция ipoly должна быть символьной (т.е подстраивается под другие выражения, а SR - символьные выражения)

## Численно-аналитический метод

Теорема. Если многочлен $f\in \mathbb{Z}[x]$ делится на многочлен $g \in \mathbb{Q}[x]$, то он делится и на $lc(f) \prod (x-x_i) \in \mathbb{Z}[x]$, где $x_i$ --- комплексные корни $g$. 

### Численно-аналитический алгоритм

Дано: $f\in \mathbb{Z}[x]$, ε > 0

Находим: 

1. Старший коэффициент:
$c = lc(c)\in \mathbb{Z}$
2. Комплексные корни:
$x_1...x_2\in \mathbb{C}$
3. Образвуем всевозможные многочлены:
$c(x-x_i 1)...(x-x_i r)$

Если один из них имеет коэффициенты, которые отличаются от целый чисел менее чем на ε > 0, то заменить эти коэффициенты на эти числа и поделить на f на этот многочлен над Q[x]. Если остаток равен нулю, то многочлен f не простой. В противном случае многочлен простой.

Теорема. Если многочлен $f\in \mathbb{Z[x]}$ делился на многолчен $g\in \mathbb{Q[x]}$, то он делится и на $lc(f) П(x-x_i)\in \mathbb{Z[x]}$, где $x_i$ - комплексные корни $g$.

3.) Напишите функцию, которая позволяет выяснить, является ли комплексное число целым с точностью до заданного $\varepsilon>0$. 

In [12]:
def is_integer(a,eps): # данная функция проверяет является ли число целым
    b=floor(a.real())
    if abs(a-b)<eps or abs(a-b-1)<eps:
        return True
    else:
        return False
    
def almost_integer(a,eps):
    b=floor(a.real()) # округляем заданное число и делаем его вещественными
    if abs(a-b)<eps: # возвращаем абсолютную величину числа (если компл.числ., то меняем на плавающее)
        return b # абсолютное значение a + bj вычисляется как math.sqrt(a^2 + b^2)
    elif abs(a-b-1)<eps:
        return b+1
    else:
        return a

In [19]:
almost_integer(1.03+0.01*i,0.1)

1

In [18]:
almost_integer(1.98+0.01*i,0.1)

2

In [20]:
almost_integer(1.6+0.01*i,0.1)

1.60000000000000 + 0.0100000000000000*I

In [21]:
is_integer(1.09+0.01*i,0.1)

True

In [22]:
is_integer(1.1+0.01*i,0.1)

False

4.) В Sage имеется численный способ отыскания комплексных корней многочлена. Допустим, для простоты, что его корни -- однократные. 

In [23]:
def alt_Kronecker(f,eps=10^-9):
    x=f.variables()[0]
    R=Combinations(f.roots(CC,False))
    for r in R[1:]:
        g=CC[x](prod([x-xi for xi in r])*(f).lc())
        if prod([is_integer(a,eps) for a in g.coefficients()]):
            g=QQ[x](sum([almost_integer(a,eps)*x^m for (a,m) in zip(g.coefficients(),g.exponents())]))
            [u,r]=f.quo_rem(g)
            if r==0:
                ans=[g,u]
                break
    return ans

In [29]:
f=QQ[x](-16*x^4 - 7*x^4 + 2*x^2 + 1)
alt_Kronecker(f)

[-23*x^4 + 2*x^2 + 1, 1]

In [28]:
var("t")
f=QQ[t](t^2-1)
alt_Kronecker(f)

[t + 1, t - 1]

5.) Что представляет собой список R? Почему выкидывается его первый элемент? 

В списке R представлены все множества, которые можно составить из корней многочлена (0, 1, 2..x), где все эти элементы не повторяются.

6.) Зачем произведение $\prod (x-x_i)$ домножается на $lc(f)$?

Чтобы образовать все многочлены, которые могут получиться.

7.) Как устроен список zip(g.coefficients(),g.exponents())?

In [52]:
eps = 10^(-9)
f = QQ[x](-14*x^4 - 7*x^2 + 2*x + 1)
x = f.variables()[0]
R = Combinations(f.roots(CC, False))
for r in R[1:]:
    g=CC[x](prod([x-xi for xi in r])*(f).lc())
    if prod([is_integer(a,eps) for a in g.coefficients()]):
        g=QQ[x](sum([almost_integer(a,eps)*x^m for (a,m) in zip(g.coefficients(),g.exponents())]))
        [u,r]= f.quo_rem(g)
        if r == 0:
            ans = [g,u]
            break
L = zip(g.coefficients(),g.exponents())
zip_L = list(L)
print(zip_L)

[(1, 0), (2, 1), (-7, 2), (-14, 4)]


zip объединяет элементы из коэффициентов и степеней заданной переменной.

## Сравнение методов

8.) Какой из двух методов быстрее?

In [53]:
g=sum([(n+1)*x^n for n in range(4)])
h=sum([(-1)^n*(n+1)*x^n for n in range(4)])
f=QQ[x](g*h)
f

-16*x^6 - 7*x^4 + 2*x^2 + 1

In [42]:
%timeit Kronecker(f)

1.31 s ± 61.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [43]:
%timeit alt_Kronecker(f)

11.3 ms ± 713 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [44]:
g=sum([(n+1)*x^n for n in range(10)])
h=sum([(-1)^n*(n+1)*x^n for n in range(5)])
f=QQ[x](g*h)
f

50*x^13 + 5*x^12 + 34*x^11 + 10*x^10 + 18*x^9 + 15*x^8 + 12*x^7 + 9*x^6 + 6*x^5 + 3*x^4 + 2*x^2 + 1

In [46]:
alt_Kronecker(f)

[50*x^4 - 40*x^3 + 30*x^2 - 20*x + 10,
 x^9 + 9/10*x^8 + 4/5*x^7 + 7/10*x^6 + 3/5*x^5 + 1/2*x^4 + 2/5*x^3 + 3/10*x^2 + 1/5*x + 1/10]

alt_Kronecker(f) быстрее.

9.) Как модифицировать функцию alt_Kronecker, чтобы она работала с кратными корнями? 

Замечание. См. конец л.р. № 2. 

In [123]:
def alt_Kronecker(f,eps=10^-9):
    x=f.variables()[0]
    g=f.gcd(diff(f,x))
    if g.degree()>0:
        (u,r)=f.quo_rem(g)
        ans=[g,u]
    else:
        R=Combinations(f.roots(CC,False))
        for r in R[1:]:
            g=CC[x](prod([x-xi for xi in r])*(f).lc())
            if prod([is_integer(a,eps) for a in g.coefficients()]):
                g=QQ[x](sum([almost_integer(a,eps)*x^m for (a,m) in zip(g.coefficients(),g.exponents())]))
                (u,r)=f.quo_rem(g)
                if r==0:
                    ans=[g,u]
                    break
    return ans

In [55]:
f=QQ[x](-16*x^6 - 7*x^4 + 2*x^2 + 1)
alt_Kronecker(f)

[-16*x^3 - 12*x^2 - 8*x - 4, x^3 - 3/4*x^2 + 1/2*x - 1/4]

In [54]:
f=QQ[x]((x^2-1)^2*(x+5))
alt_Kronecker(f)

[x + 5, x^4 - 2*x^2 + 1]